In [1]:
import pandas as pd
import numpy as np
import xlsxwriter as xl

In [2]:
# Processing the excel files 
df = pd.read_excel('Nov_2019_V1.xlsx', sheet_name = 'Job Ledger Entries')

resoruces = pd.read_csv("Resources.csv")
resoruces.rename(columns = {"Id" : "No."}, inplace = True)
resourcesIDs = resoruces[["No.","Name"]]

In [3]:
# Arranging the columns details
array = df.columns
for s in array:
    if "help" in s.lower():
        df.rename(columns = {s : "Helpdesk"}, inplace = True)
    elif "bill" in s.lower():
        df.rename(columns = {s : "Billable/Non-Billable"}, inplace = True)

# Creating the first graph dataset
help = df[df["Helpdesk"]== True].groupby("No.").count().index
help_quantiy = df[df["Helpdesk"]== True].groupby("No.").sum()["Quantity"]

otherBillable = df[(df["Billable/Non-Billable"] == "Billable") & (df["Helpdesk"] != True)].groupby("No.").sum()["Quantity"]
nonBillable = df[(df["Billable/Non-Billable"] == "Non-Billable")].groupby("No.").sum()["Quantity"]

In [4]:
billable_df = pd.merge(help_quantiy,otherBillable,'inner',on="No.")

first_df = pd.merge(billable_df,nonBillable, 'inner',on="No.")
first_df["Total"] = first_df["Quantity_x"] + first_df["Quantity_y"] + first_df["Quantity"]
first_df["ShiftDetails"] = ""

first_df.rename(columns={"Quantity_x":"Helpdesk", "Quantity_y":"Other_Billable", "Quantity":"NonBillable"}, inplace =True)
first_df.sort_values(by="Helpdesk",ascending = False, inplace = True)

first_df = pd.merge(first_df,resourcesIDs,'inner', on = "No.")

name = first_df["Name"]
first_df.insert(1,"Names",name)
first_df.drop("Name",axis = 1, inplace = True)
first_df.set_index("No.", inplace = True)

In [5]:
# Creating the Second graph dataset
second_df = pd.DataFrame(columns = ["Helpdesk","Other_Billable","NonBillable"], index = first_df.index)
second_df["Helpdesk"] = first_df["Helpdesk"] / first_df["Total"]
second_df["Other_Billable"] = first_df["Other_Billable"] / first_df["Total"]
second_df["NonBillable"] = first_df["NonBillable"] / first_df["Total"]
second_df_adjusted = second_df.apply(lambda x : round(x *100,2))

second_df_adjusted = pd.merge(second_df_adjusted,resourcesIDs,'inner', on = "No.")

name = second_df_adjusted["Name"]
second_df_adjusted.insert(1,"Names",name)
second_df_adjusted.drop("Name",axis = 1, inplace = True)
second_df_adjusted.set_index("No.", inplace = True)

In [6]:
# Setting up the excel writer
writer = pd.ExcelWriter("HDReport_November.xlsx", engine= "xlsxwriter")
first_df.to_excel(writer,sheet_name="sheet1")           
workbook = writer.book
sheet = writer.sheets["sheet1"]

# Setting up first dataframe Graph
chart = workbook.add_chart({'type': 'column'})
column_no = str(len(first_df.index)+2)

chartseries1_excelway = "sheet1" + "!" + "$C$2:" + "$C$" + column_no
chartseries2_excelway = "sheet1" + "!" + "$D$2:" + "$D$" + column_no
chartseries3_excelway = "sheet1" + "!" + "$E$2:" + "$E$" + column_no
chartseries4_excelway = "sheet1" + "!" + "$F$2:" + "$F$" + column_no
chartCategpory = "sheet1" + "!" + "$B$2:" + "$B$" + str(len(first_df.index)+1)
print(chartseries1_excelway)

# Configure the series of the chart from the dataframe data.
chart.add_series({ "values" : chartseries1_excelway, "name" : "helpdesk_billable","categories" : chartCategpory, 'data_labels': {'value': True}})
chart.add_series({ "values" : chartseries2_excelway, "name" : "billable", 'data_labels': {'value': True}})
chart.add_series({ "values" : chartseries3_excelway, "name" : "non_billable", 'data_labels': {'value': True}})
chart.add_series({ "values" : chartseries4_excelway, "name" : "Total", 'data_labels': {'value': True}})

chart.set_title({"name" : "Help Desk utilisation"})
chart.set_size({"x_scale": 2.5 , "y_scale" : 1.6})

# Insert the chart into the worksheet.
chart_cell = "B" + str(len(first_df.index)+4)
sheet.insert_chart(chart_cell, chart)

second_df_adjusted.to_excel(writer, sheet_name="sheet1",
             startcol=0,startrow=35)

# Setting up Second dataframe Graph
chart2 = workbook.add_chart({'type': 'column'})

column_no2 = str(len(first_df.index)+36)
chart2series1_excelway = "sheet1" + "!" + "$C$37:" + "$C$" + column_no2
chart2series2_excelway = "sheet1" + "!" + "$D$37:" + "$D$" + column_no2
chart2series3_excelway = "sheet1" + "!" + "$E$37:" + "$E$" + column_no2
chart2Categpory = "sheet1" + "!" + "$B$37:" + "$B$" + column_no2

chart2.add_series({ "values" : chart2series1_excelway, "name" : "helpdesk_billable","categories" : chart2Categpory, 'data_labels': {'value': True}})
chart2.add_series({ "values" : chart2series2_excelway, "name" : "billable%",'data_labels': {'value': True}})
chart2.add_series({ "values" : chart2series3_excelway, "name" : "non_billable%", 'data_labels': {'value': True}})

chart2.set_title({"name" : "Help Desk utilisation % against Total"})
chart2.set_size({"x_scale": 2.25 , "y_scale" : 1.5})

chart_cell = "B" + str(len(first_df.index)+42)
sheet.insert_chart(chart_cell, chart2)
writer.save()

sheet1!$C$2:$C$11
